In [1]:
import sys
import keras
import cv2
import numpy
import matplotlib
import skimage
from keras.models import Sequential
from keras.layers import Conv2D
from keras.optimizers import Adam
from skimage.measure import compare_ssim as ssim
from matplotlib import pyplot as plt
import cv2
import numpy as np
import math
import os


In [2]:
def prepare_images(path, factor):
    
    # loop through the files in the directory
    for file in os.listdir(path):
        
        # open the file
        img = cv2.imread(path + '/' + file)
        
        # find old and new image dimensions
        h, w, _ = img.shape
        
        new_height = int( h / factor )
        
        new_width = int( w / factor )
        
        # resize the image - down
        img = cv2.resize(img, (new_width, new_height), interpolation = cv2.INTER_LINEAR)
        
        # resize the image - up
        img = cv2.resize(img, (w, h), interpolation = cv2.INTER_LINEAR)
        
        # save the image
        print('Saving {}'.format(file))
        cv2.imwrite('/home/bharath/Desktop/Mini project/design project/data/images/{}'.format(file), img)

In [3]:
prepare_images('/home/bharath/Desktop/Mini project/design project/data/rdn/', 2)

Saving melc4.jpg
Saving melc26.jpg
Saving melc16.jpg
Saving melc6.jpg
Saving melc18.jpg
Saving melc9.jpg
Saving melc24.jpg
Saving melc20.jpg
Saving melc25.jpg
Saving melc15.jpg
Saving melc0.jpg
Saving melc1.jpg
Saving melc19.jpg
Saving melc23.jpg
Saving melc3.jpg
Saving melc30.jpg
Saving melc28.jpg
Saving melc31.jpg
Saving melc27.jpg
Saving melc10.jpg
Saving melc8.jpg
Saving melc7.jpg
Saving melc13.jpg
Saving melc12.jpg
Saving melc17.jpg
Saving melc11.jpg
Saving melc14.jpg
Saving melc5.jpg
Saving melc29.jpg
Saving melc21.jpg
Saving melc2.jpg
Saving melc22.jpg


In [4]:
def model():
    
    # define model type
    SRCNN = Sequential()
    
    # add model layers
    SRCNN.add(Conv2D(filters=128, kernel_size = (9, 9), kernel_initializer='glorot_uniform',
                     activation='relu', padding='valid', use_bias=True, input_shape=(None, None, 1)))
    SRCNN.add(Conv2D(filters=64, kernel_size = (3, 3), kernel_initializer='glorot_uniform',
                     activation='relu', padding='same', use_bias=True))
    SRCNN.add(Conv2D(filters=1, kernel_size = (5, 5), kernel_initializer='glorot_uniform',
                     activation='linear', padding='valid', use_bias=True))
    
    # define optimizer
    adam = Adam(lr=0.0003)
    
    # compile model
    SRCNN.compile(optimizer=adam, loss='mean_squared_error', metrics=['mean_squared_error'])
    
    return SRCNN

In [5]:
def modcrop(img, scale):
    tmpsz = img.shape
    sz = tmpsz[0:2]
    sz = sz - np.mod(sz, scale)
    img = img[0:sz[0], 1:sz[1]]
    return img


def shave(image, border):
    img = image[border: -border, border: -border]
    return img

In [6]:
def predict(image_path):
    
    # load the srcnn model with weights
    srcnn = model()
    srcnn.load_weights('3051crop_weight_200.h5')
    
    # load the degraded and reference images
    path, file = os.path.split(image_path)
    degraded = cv2.imread(image_path)
    ref = cv2.imread('/home/bharath/Desktop/Mini project/design project/data/rdn/{}'.format(file))
    
    # preprocess the image with modcrop
    degraded = modcrop(degraded, 3)
    
    # convert the image to YCrCb - (srcnn trained on Y channel)
    temp = cv2.cvtColor(degraded, cv2.COLOR_BGR2YCrCb)
    
    # create image slice and normalize  
    Y = numpy.zeros((1, temp.shape[0], temp.shape[1], 1), dtype=float)
    Y[0, :, :, 0] = temp[:, :, 0].astype(float) / 255
    
    # perform super-resolution with srcnn
    pre = srcnn.predict(Y, batch_size=1)
    
    # post-process output
    pre *= 255
    pre[pre[:] > 255] = 255
    pre[pre[:] < 0] = 0
    pre = pre.astype(np.uint8)
    
    # copy Y channel back to image and convert to BGR
    temp = shave(temp, 6)
    temp[:, :, 0] = pre[0, :, :, 0]
    output = cv2.cvtColor(temp, cv2.COLOR_YCrCb2BGR)
    
    
    
    # return images and scores
    return output

In [7]:
for file in os.listdir('/home/bharath/Desktop/Mini project/design project/data/rdn/'):
    
    # perform super-resolution
    output = predict('/home/bharath/Desktop/Mini project/design project/data/images/{}'.format(file))
    
    print('Saving {}'.format(file))
    cv2.imwrite('/home/bharath/Desktop/Mini project/design project/data/output/cnno/{}'.format(file), output)



Saving melc4.jpg
Saving melc26.jpg
Saving melc16.jpg
Saving melc6.jpg
Saving melc18.jpg
Saving melc9.jpg
Saving melc24.jpg
Saving melc20.jpg
Saving melc25.jpg
Saving melc15.jpg
Saving melc0.jpg
Saving melc1.jpg
Saving melc19.jpg
Saving melc23.jpg


Saving melc3.jpg
Saving melc30.jpg
Saving melc28.jpg
Saving melc31.jpg
Saving melc27.jpg
Saving melc10.jpg
Saving melc8.jpg
Saving melc7.jpg
Saving melc13.jpg
Saving melc12.jpg
Saving melc17.jpg


Saving melc11.jpg
Saving melc14.jpg
Saving melc5.jpg
Saving melc29.jpg
Saving melc21.jpg
Saving melc2.jpg
Saving melc22.jpg
